# ***Subset selection:***
This notebook aims at demonstrating the use cases for the functions in spear library for subset selection. Subset selection is selecting a small subset of unlabeled data(or the data labeled by LFs, in case of supervised subset selection) so that it can be labeled and use that small labeled data for effective training of Cage or JL algorithms. Finding the best subset makes best use of the labeling efforts.
